In [1]:
%matplotlib qt
import hyperspy.api as hs
import scipy.misc
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.signal import find_peaks

C:\Users\chhe\Anaconda3\envs\hspy_environment\lib\site-packages\pyUSID\viz\__init__.py:18: FutureWarning: Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID
  FutureWarning)


# Define Variables

In [2]:
wavelength_TEM_200kV = 0.0251*10**-10 #m 
d_Al_220 = 1.432 * 10**-10
d_Al_200 = 2.021*10**-10

# Define Functions

##### make_si: 
Takes Theta values (min 3 theta values) and converts them into the deviation parameter s

##### get_thichness_plot:
Main function. Takes the Si_list (min 3 s_i values) and tries to iteratively arrange them. Condition for termination is either best fit (highest R2 value) or exceeding of kStop-iterations. Starts with iterator kStart.

In [3]:
def make_si(THETA_list, D_KIK, D_HKL, WAVELENGTH_TEM):
    if len(THETA_list)<3:
        print('Not enough Theta values!')
    SI_list = [(WAVELENGTH_TEM*THETA)/(D_KIK*D_HKL**2) for THETA in THETA_list]
    return np.array(SI_list)

def get_thickness_plot(SI_list, kStart = 1, kStop = 6, D_EXCT= None):
    RSQUARE_list = []
    DATA_list = [] #Saving interesting stuff per Iteration to make plots
    if len(SI_list)<3:
        print('Not enough s_i values!')
    for i in range(kStop):
      
        #while RSQUARE_list[-1]>RSQUARE_list[-2]:
        NK_list = np.arange(start=kStart, stop= len(SI_list)+kStart, step=1) #creating the necessary n_k
        SIsquared_over_NKsquared= np.array([S**2/N**2 for (S,N) in zip(SI_list,NK_list)]) #creating Yvalues
        ONE_over_NKsquared = np.array([1/N**2 for N in NK_list]) #creating Xvalues
        SLOPE, Yintercept, RSQUARE, p_value, std_err = linregress(ONE_over_NKsquared, SIsquared_over_NKsquared) #Getting all the necessary Data
        
        # Getting Extinction length and Thickness T
        D_EXCT_G = np.sqrt(np.abs(1/SLOPE)) * 10**9 # in nm
        T = np.sqrt(1/Yintercept)*10**9 # in nm  

        ############
        #### Saving Results
        RSQUARE_list.append(np.abs(RSQUARE))
        DATA_list.append([SLOPE, Yintercept, T, D_EXCT_G,np.abs(RSQUARE), SIsquared_over_NKsquared, ONE_over_NKsquared])
        kStart+=1
    ###############
    # Comparing Measured Exctinction length with a given one
    if D_EXCT != None:
        print(D_EXCT_G)
        print(D_EXCT_G/D_EXCT)

    ############
    ############
    #  Plotting
  
    plt.rcParams.update({'font.size': 18})
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(20,5))
    #ax1.set_aspect(aspect=0.5)  # Doing Kelly's Plot
    BEST_fittedITERATION= (RSQUARE_list.index(max(RSQUARE_list)))
    Best_THICKNESS = DATA_list[BEST_fittedITERATION][2]
    Best_SLOPE =DATA_list[BEST_fittedITERATION][0]
    print(type(Best_SLOPE))
    ax1.scatter(DATA_list[BEST_fittedITERATION][-1],DATA_list[BEST_fittedITERATION][-2], c= 'r', s = 24, label=r'Thickness= '+str(round(Best_THICKNESS,2))+' nm \n $R^2= $'+str(round(DATA_list[BEST_fittedITERATION][4],4)))
    ax1.plot(DATA_list[BEST_fittedITERATION][-1],DATA_list[BEST_fittedITERATION][1] + DATA_list[BEST_fittedITERATION][0]* DATA_list[BEST_fittedITERATION][-1], c='g', lw=3, alpha= 0.4, label= 'Perfect Condition')
    ax1.legend(loc= 'upper right')
    ax1.set_xlabel(r'$\frac{1}{n_k^2}$')
    y_label= ax1.set_ylabel(r'$\frac{s_i^2}{n_k^2}$')
    y_label.set_rotation(0)
    ax1.set_yticklabels([])
    ax1.set_xticklabels([])
    if Best_SLOPE >0:
        ax1.text(.5,.5,'Wrong Slope',size = 100, bbox=dict(boxstyle="round",ec=(1., 0.5, 0.5),fc=(1., 0.8, 0.8)))
    #ax2.set_aspect(aspect= 1)   # Plotting the Rsquared
    ax2.scatter(np.arange(start= 1, stop=len(RSQUARE_list)+1,step=1, dtype= int), RSQUARE_list, marker= '+',s = 200)
    ax2.scatter(BEST_fittedITERATION+1, DATA_list[BEST_fittedITERATION][4], s= 200, facecolors = 'none', edgecolors= 'r',linewidth= 3)
    ax2.set_xlim([0,kStop+1])
    #ax2.set_xticklabels(np.arange(start= 1, stop=len(RSQUARE_list)+1,step=1, dtype= int))
    ax2.set_xlabel('Number of Iterations')
    ylabel = ax2.set_ylabel(r'$R^2$')
    ylabel.set_rotation(90)
    
    return fig, RSQUARE_list

# Load Image

In [15]:
CBED_path = "C:/Users/chhe/OneDrive - NTNU/Archive/(CBED)Thicknessdetermination/20201030_CBED/CBED_Spot1Alpha1_L120cm_G200.dm3"
CBED_img = hs.load(CBED_path)
CBED_img.plot()

# Choose ROI
Arrange the Line-Scan tool in a way to extract the information of the KM fringes and the distant of the Kikuchi lines.

In [8]:
roi = hs.roi.Line2DROI(10,10,5,5, linewidth = 5)
im_roi = roi.interactive(CBED_img,color = 'red')



Line2DROI(x1=10, y1=10, x2=5, y2=5, linewidth=5)


# Get ROI and distances
Span the interactive (green) widget to either extract the distance of Kik.lines or the Theta values.

First, one wants to extract the distance between the Kikuchi lines. Afterwards, one wants to extract the distances of the KM lines (min 3) to the Kikuchi line in the gspot. The reference (green edge on the Kikuchi line) should not be changed.

In [9]:
im_roi.plot()
span= hs.roi.SpanROI(0, 5)
span.add_widget(im_roi)
theta_list = []

#### d_kik

In [10]:
R = span.right
L = span.left
d_kik= span.right-span.left
d_kik

8.053430378446832

<class 'numpy.ndarray'>


#### theta_1

In [11]:
theta1= span.right-span.left
theta_list = [theta1]
theta1

1.32315443816504

#### theta_2

In [12]:
theta2= span.right-span.left
theta_list = [theta1, theta2]
theta2

2.4809145715594503

#### theta_3 

In [13]:
theta3= span.right-span.left
theta_list = [theta1, theta2, theta3]
theta3

3.8040690097244907

#### theta_4 

In [13]:
theta4= span.right-span.left
theta_list = [theta1, theta2, theta3, theta4]
theta4

2.8078136989665694

#### theta_5 

In [ ]:
theta5= span.right-span.left
theta_list = [theta1, theta2, theta3, theta4, theta5]
theta5

# Run Functions

It is important to set/use the correct d-plane spacing.

In [14]:
si_list = make_si(theta_list,d_kik, d_Al_220, wavelength_TEM_200kV)
thickness_fig, rsquares = get_thickness_plot(si_list[:3], kStart = 1, kStop = 5)

<class 'numpy.float64'>
